# BioExplorer - Vasculature
![](../bioexplorer_vasculature_banner.png)

### Connect to back-end

In [ ]:
from bioexplorer import BioExplorer, Vector3
import seaborn as sns

url = 'r2i2n8.bbp.epfl.ch:5000'
be = BioExplorer(url)
core = be.core_api()
status = be.reset_scene()

In [ ]:
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from tqdm.notebook import tqdm
import numpy as np

db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_schema = 'wholebrain_ml20200201_point_vasculature_lps_registrered_25um'

db_connection_string = 'postgresql+psycopg2://%s:%s@%s:5432/%s' % (db_user, db_password, db_host, db_name)
print('Connection string: ' + db_connection_string + ', schema: ' + db_schema)

engine = create_engine(db_connection_string)
conn = engine.connect()

population_name = db_schema

### Load vasculature by region

In [ ]:
with Session(engine) as session:
    data = session.execute('SELECT DISTINCT(region_guid) FROM %s.node' % db_schema)
    regions = data.all()

In [ ]:
from tqdm import tqdm

for region in tqdm(regions):
    region_guid = region[0]
    try:
        assembly_name = 'Vasculature_Region_%d' % region_guid
        be.remove_assembly(assembly_name)
        vasculature_assembly = be.add_assembly(assembly_name)
        vasculature_model = be.add_vasculature(
            assembly_name=assembly_name,
            population_name=population_name,
            representation=be.VASCULATURE_REPRESENTATION_SEGMENT,
            radius_multiplier=1.0/25.0,
            sql_filter='region_guid=%d' % region_guid
        )
    except RuntimeError as e:
        print(e)

### Set materials

In [ ]:
def set_materials(model_id, color):
    material_ids = be.get_material_ids(model_id)['ids']
    nb_material_ids = len(material_ids)
    palette = list()
    for _ in material_ids:
        palette.append(color)
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        diffuse_colors=palette, specular_colors=palette)

model_ids = be.get_model_ids()['ids']
palette = sns.color_palette('Set3', len(model_ids))
i = 0
for model_id in model_ids:
    set_materials(model_id, palette[i])
    i += 1
be.reset_camera()
core.set_renderer()

### Rendering settings

In [ ]:
status = core.set_renderer(
    current='bio_explorer',
    subsampling=4, max_accum_frames=64)
params = core.BioExplorerRendererParams()
params.gi_distance = 1000.0
params.shadows = 1.0
params.soft_shadows = 1.0
status = core.set_renderer_params(params)